### Task #1

- Tokenize the text
- Remove stopwords and compute the size of the vocabulary.
- Compute the reduction rate of the vocabulary size after removing stopwords.
- Remove word variations with stemming and compute the size of the vocabulary.
- Compute the reduction rate of the vocabulary size after stemming.

Describe which procedures (and which libraries) you used and why they are appropriate.

In [ ]:
### Code ###

### Task #2

- Describe how you ended up representing the FakeNewsCorpus dataset (for instance with a Pandas dataframe). Argue for why you chose this design.
- Did you discover any inherent problems with the data while working with it?
- Report key properties of the data set - for instance through statistics or visualization.

The exploration can include (but need not be limited to):

- counting the number of URLs in the content
- counting the number of dates in the content
- counting the number of numeric values in the content
- determining the 100 more frequent words that appear in the content
- plot the frequency of the 10000 most frequent words (any interesting patterns?)
- run the analysis in point 4 and 5 both before and after removing stopwords and applying stemming: do you see any difference?

In [ ]:
### Code ###

### Task #3

Apply your data preprocessing pipeline to a larger proportion of the FakeNewsCorpus https://github.com/several27/FakeNewsCorpus/releases/tag/v1.0

You may find it challenging to run your data processing pipeline on the entire FakeNewsCorpus. At a minimum, you should be able to process 10% of the data using your pipeline,

In [ ]:
### Code ###

### Task #4

Split the resulting dataset into a training, validation, and test splits. A common strategy is to uniformly at random split the data 80% / 10% / 10%. You will use the training data to train your baseline and advanced models, the validation data can be used for model selection and hyperparameter tuning, while the test data should only be used in Part 4.

In [ ]:
### Code ###